In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
Folder_A = 'A.The_full_year_MILP'
Folder_B = 'B.Operation_cost'
Folder_D = 'D.Representative_days_based_on_RES_and_Demand'
Folder_E = 'E.Representative_days_based_on_Line_Benefits_OptModel'
Folder_K = 'K.Investments_per_hour'
Folder_L = 'L.Cont_Investments_per_hour'

category_dict = { "FYMILP": Folder_A,
                 "OPC":Folder_B,
                 "R&D": Folder_D ,
                 "OPT_LB": Folder_E,
                 "HI": Folder_K,
                  "CHI": Folder_L,
                }

In [3]:
CaseName_Base = "9n_mod1"
cm = "CHI"
nbc = 10
folder_name = category_dict[cm]

In [66]:
destination_folder = f"../Y.FYTS_from_ByStages/{CaseName_Base}/{cm}"

filename = f"NetDemand_nc{nbc}.csv"

# Read input data
df_duration = pd.read_csv(
    f"{folder_name}/{CaseName_Base}_ByStages_nc{nbc}/2.Par/oT_Data_Duration_{CaseName_Base}_ByStages_nc{nbc}.csv")
df_ts_bs_dem = pd.read_csv(
    f"{folder_name}/{CaseName_Base}_ByStages_nc{nbc}/2.Par/oT_Data_Demand_{CaseName_Base}_ByStages_nc{nbc}.csv")
df_ts_bs_vgen = pd.read_csv(
    f"{folder_name}/{CaseName_Base}_ByStages_nc{nbc}/2.Par/oT_Data_VariableMaxGeneration_{CaseName_Base}_ByStages_nc{nbc}.csv")

# Values summed per hour
vargen_ph = df_ts_bs_vgen.rename(columns = {"Unnamed: 2": "LoadLevel"}).iloc[:,2:].set_index("LoadLevel").sum(axis=1)
dem_ph = df_ts_bs_dem.rename(columns = {"Unnamed: 2": "LoadLevel"}).iloc[:,2:].set_index("LoadLevel").sum(axis=1)
net_dem_ph = dem_ph- vargen_ph


# Create mapping of load level to stage name
load_level_to_stage_map = df_duration.set_index("LoadLevel")["Stage"].to_dict()
all_load_levels = df_duration.LoadLevel

# Create dataframe that that wil hold all timeseries
frame_values = pd.DataFrame({"LoadLevel": all_load_levels})

# Initialize a DataFrame to hold all time series with MultiIndex
frame_values = pd.DataFrame(index=all_load_levels)


stage_to_repr_load_level_map = df_duration[df_duration["Duration"] == 1].set_index("Stage")["LoadLevel"].to_dict()

frame_values["NetDemand"] = frame_values.index.map(load_level_to_stage_map).map(stage_to_repr_load_level_map).map(repr_load_level_to_value_map)


In [67]:
frame_values

,NetDemand
LoadLevel,
01-01 00:00:00+01:00,109.217946
01-01 01:00:00+01:00,109.217946
01-01 02:00:00+01:00,109.217946
01-01 03:00:00+01:00,109.217946
01-01 04:00:00+01:00,109.217946
...,...
12-30 19:00:00+01:00,815.717655
12-30 20:00:00+01:00,815.717655
12-30 21:00:00+01:00,815.717655


In [63]:
repr_load_level_to_value_map = {ll : net_dem_ph.loc[ll] for ll in stage_to_repr_load_level_map.values()}
repr_load_level_to_value_map

{'01-07 16:00:00+01:00': 1076.0043629129998,
 '01-10 09:00:00+01:00': 1223.141292198,
 '01-14 20:00:00+01:00': 1033.2058250219998,
 '01-29 10:00:00+01:00': 1110.618033119,
 '01-29 11:00:00+01:00': 971.6015571579997,
 '01-30 08:00:00+01:00': 1561.5618548509997,
 '04-13 11:00:00+02:00': 109.21794632900014,
 '07-01 17:00:00+02:00': 555.0518304809999,
 '08-19 12:00:00+02:00': 20.802763302999892,
 '09-25 20:00:00+02:00': 815.7176551650001}

In [53]:
df_ts_bs_vgen.rename(columns = {"Unnamed: 2": "LoadLevel"}).iloc[:,2:].set_index("LoadLevel").sum(axis=1)

LoadLevel
01-01 00:00:00+01:00    701.378167
01-01 01:00:00+01:00    717.589404
01-01 02:00:00+01:00    688.056535
01-01 03:00:00+01:00    678.509940
01-01 04:00:00+01:00    673.851645
                           ...    
12-30 19:00:00+01:00    550.814706
12-30 20:00:00+01:00    539.784252
12-30 21:00:00+01:00    546.047033
12-30 22:00:00+01:00    573.927427
12-30 23:00:00+01:00    583.265045
Length: 8736, dtype: float64

In [43]:
df_ts_bs_dem.iloc[: , 2:]

,LoadLevel,Node_1,Node_2,Node_3,Node_4,Node_5,Node_6,Node_7,Node_8,Node_9
0,01-01 00:00:00+01:00,0.661916,142.835721,26.574231,100.060335,124.314207,45.129818,131.860811,8.927233,41.454057
1,01-01 01:00:00+01:00,0.584187,118.993998,25.689395,88.790906,106.156962,38.321117,131.015492,7.437125,36.489574
2,01-01 02:00:00+01:00,0.502973,97.975699,23.151571,66.910232,87.383347,29.662053,131.706087,6.123481,28.673952
3,01-01 03:00:00+01:00,0.400147,88.651721,20.565129,48.416809,71.755789,25.071691,131.184388,5.540733,20.264344
4,01-01 04:00:00+01:00,0.382371,89.300345,18.095369,36.312607,70.767028,22.698916,130.922658,5.581272,15.287356
...,...,...,...,...,...,...,...,...,...,...
8731,12-30 19:00:00+01:00,1.310587,320.363320,62.346859,223.269556,303.459699,118.136297,143.506500,20.022707,116.602821
8732,12-30 20:00:00+01:00,1.122713,281.035684,56.143289,226.753126,265.347462,111.371704,139.695355,17.564730,102.959872
8733,12-30 21:00:00+01:00,0.956798,219.640505,49.093776,223.285609,239.382858,96.366249,137.118642,13.727532,88.010151
8734,12-30 22:00:00+01:00,0.918457,163.205395,41.169152,194.469902,221.880506,81.516356,133.911715,10.200337,69.027569


In [31]:
df_ts_bs_dem.rename(columns = {"Unnamed: 2": "LoadLevel"},inplace=True)
df_ts_bs_dem

,Unnamed: 0,Unnamed: 1,LoadLevel,Node_1,Node_2,Node_3,Node_4,Node_5,Node_6,Node_7,Node_8,Node_9
0,2030.0,sc01,01-01 00:00:00+01:00,0.661916,142.835721,26.574231,100.060335,124.314207,45.129818,131.860811,8.927233,41.454057
1,2030.0,sc01,01-01 01:00:00+01:00,0.584187,118.993998,25.689395,88.790906,106.156962,38.321117,131.015492,7.437125,36.489574
2,2030.0,sc01,01-01 02:00:00+01:00,0.502973,97.975699,23.151571,66.910232,87.383347,29.662053,131.706087,6.123481,28.673952
3,2030.0,sc01,01-01 03:00:00+01:00,0.400147,88.651721,20.565129,48.416809,71.755789,25.071691,131.184388,5.540733,20.264344
4,2030.0,sc01,01-01 04:00:00+01:00,0.382371,89.300345,18.095369,36.312607,70.767028,22.698916,130.922658,5.581272,15.287356
...,...,...,...,...,...,...,...,...,...,...,...,...
8731,2030.0,sc01,12-30 19:00:00+01:00,1.310587,320.363320,62.346859,223.269556,303.459699,118.136297,143.506500,20.022707,116.602821
8732,2030.0,sc01,12-30 20:00:00+01:00,1.122713,281.035684,56.143289,226.753126,265.347462,111.371704,139.695355,17.564730,102.959872
8733,2030.0,sc01,12-30 21:00:00+01:00,0.956798,219.640505,49.093776,223.285609,239.382858,96.366249,137.118642,13.727532,88.010151
8734,2030.0,sc01,12-30 22:00:00+01:00,0.918457,163.205395,41.169152,194.469902,221.880506,81.516356,133.911715,10.200337,69.027569
